In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#pd.options.mode.copy_on_write = True

In [3]:
def load_data() -> pd.DataFrame:
    dataset = "country_vaccination_stats.csv"
    data = pd.read_csv(dataset)
    return data

In [12]:
def fill_missing_data(pd_data: pd.DataFrame) -> pd.DataFrame:
    min_vaccinations = pd_data.groupby('country')['daily_vaccinations'].transform('min')
    pd_data['daily_vaccinations'] = pd_data['daily_vaccinations'].fillna(min_vaccinations).fillna(0)
    
    return pd_data


# Example usage
data = load_data()
filled_data = fill_missing_data(data)

,country,date,daily_vaccinations,vaccines
750,Kuwait,12/28/2020,0.0,Pfizer/BioNTech


In [5]:
def get_top3_medians(pd_data: pd.DataFrame) -> pd.DataFrame:
    top3 = pd_data.groupby('country')['daily_vaccinations'].median().sort_values(ascending=False).head(3)
    return top3

# Example usage
top3 = get_top3_medians(filled_data)
top3


country
United States    378253.0
China            276786.0
India            173922.0
Name: daily_vaccinations, dtype: float64

In [11]:
def get_vaccination_by_date(pd_data: pd.DataFrame) -> pd.DataFrame:
    pd_data['date'] = pd.to_datetime(pd_data['date'])
    pd_data = pd_data.groupby('date')['daily_vaccinations'].sum().reset_index()
    return pd_data

# Example usage
vaccination_by_date = get_vaccination_by_date(filled_data)
vaccination_by_date[vaccination_by_date['date'] == '2021-01-06']

,date,daily_vaccinations
24,2021-01-06,1485255.0


In [14]:
WITH MedianValues AS (
    SELECT
        country,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY daily_vaccinations) AS median_vaccination
    FROM
        vaccination_data
    WHERE
        daily_vaccinations IS NOT NULL
    GROUP BY
        country
)

-- Update the main table with the median values
UPDATE vaccination_data
SET daily_vaccinations = COALESCE(
    (SELECT median_vaccination FROM MedianValues WHERE vaccination_data.country = MedianValues.country),
    0
)
WHERE
    daily_vaccinations IS NULL;

NameError: name 'SELECT' is not defined